In [1]:
from delta import *
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from dotenv import load_dotenv
import os

load_dotenv()

spark = pyspark.sql.SparkSession.builder.appName("DeltaLake") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0,org.apache.hadoop:hadoop-aws:3.2.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_USER")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_PASSWORD")) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("com.amazonaws.services.s3.enableV4", True) \
    .config("spark.driver.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true") \
    .getOrCreate()

#spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()


21/09/06 19:55:23 WARN Utils: Your hostname, DESKTOP-58VHONS resolves to a loopback address: 127.0.1.1; using 172.18.134.119 instead (on interface eth0)
21/09/06 19:55:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/smikic/.cache/pypoetry/virtualenvs/realestatedataengineering-la76Cphi-py3.8/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/smikic/.ivy2/cache
The jars for the packages stored in: /home/smikic/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-23fb0957-2781-4d21-b306-187e4b0e55e6;1.0
	confs: [default]
	found io.delta#delta-core_2.12;0.7.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 755ms :: artifacts dl 36ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	com.ibm.icu#icu4j;5

In [2]:
print(os.getenv("MINIO_USER"))
print(os.getenv("MINIO_PASSWORD"))

admin
superadmin


In [8]:
#df = spark.read.csv("../data/apt_data.csv")
# Bucket needs to be created first
#df.write.format("delta").save("s3a://apt/delta")
df = spark.read.format("delta").load("s3a://apt/delta")
df.tail(2)

[Row(_c0='956', _c1='Büroräumlichkeiten / Top 6', _c2='Preis auf Anfrage', _c3='N/A', _c4='110\xa0m', _c5='Bludenz', _c6='Bludenz', _c7=' , Wichnerstraße '),
 Row(_c0='958', _c1='Büroräumlichkeiten / Top 9', _c2='Preis auf Anfrage', _c3='N/A', _c4='110\xa0m', _c5='Bludenz', _c6='Bludenz', _c7=' , Wichnersstraße ')]